<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/transfer_learning_from_TADNE_512_tpu_aydao_stylegan2_surgery_model_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
#!gcloud init

In [2]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-aydao"):
    %cd "/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-aydao
    %cd colab-sg2-aydao
    !git clone --branch model-release https://github.com/aydao/stylegan2-surgery.git
#    !git clone --branch tpu https://github.com/shawwn/stylegan2.git
#    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-surgery
    !mkdir downloads
    !mkdir datasets

/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery


In [ ]:
%cd "/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
#!git fetch origin
#!git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/sq-512.zip"

!unzip {zip_path} -d /content/

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
!gcloud auth login

In [9]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#update this to the path to your image folder
dataset_path = "/content/sq-512"
#give your dataset a name
dataset_name = "sq-512"

#you don't need to edit anything here
#!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}
#!python dataset_tool.py create_from_images gs://train_with_tpu/datasets/{dataset_name} {dataset_path}
!python dataset_tool.py create_aydao gs://train_with_tpu/datasets_aydao {dataset_path} {dataset_name} 512 512


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
  import os
  import tensorflow.compat.v1 as tf
  tf.disable_eager_execution()
  import pprint
  assert 'COLAB_TPU_ADDR' in os.environ, 'Did you forget to switch to TPU?'
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(tpu_address) as sess:
    devices = sess.list_devices()
  pprint.pprint(devices)
  device_is_tpu = [True if 'TPU' in str(x) else False for x in devices]
  assert True in device_is_tpu, 'Did you forget to switch to TPU?'


In [ ]:
print(tpu_address)

grpc://10.101.159.26:8470


In [ ]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

In [ ]:
!gsutil cp -r /content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery/datasets/* gs://train_with_tpu/datasets/


In [21]:
!gsutil cp -r /content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/* gs://train_with_tpu/datasets/


Copying file:///content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/sq-512/sq-512-r08.tfrecords [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/sq-512/sq-512-r07.tfrecords [Content-Type=application/octet-stream]...
Copying file:///content/driv

In [11]:
GCP_PROJECT_ID = 'encoded-phalanx-326615'
PROJECT_NUMBER = '0'


In [12]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com


Operation "operations/acf.p2-174339478235-ce99fa28-c4f9-4fce-a29c-915933ad2cd2" finished successfully.


In [14]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gsutil cp -r /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl gs://train_with_tpu/models/

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
!python copy_weights_portable.py /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl ./model.ckpt-1024.pkl

In [ ]:
!python convert_pkl_to_ckpt.py --ckpt_model_dir /content/drive/MyDrive/models/sq-256 --input_pkl ./network-copyover.pkl
#!python convert_pkl_to_ckpt.py --ckpt_model_dir /content/drive/MyDrive//models/sq-256 --input_pkl /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl


In [ ]:
!gsutil cp -r /content/drive/MyDrive/models/sq-256/* gs://train_with_tpu/networks_aydao/sq-256

In [ ]:
#!python convert_pkl_to_ckpt.py --tpu_address {tpu_address} --ckpt_model_dir gs://train_with_tpu/models/sq-256 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
!chmod 777 -R ./tpu_config.sh
#!export TPU_HOST={tpu_address} # placeholder -- refer to shawwn's tpu code
#!export TPU_NAME={tpu_address} # placeholder -- refer to shawwn's tpu code
#!export WARM_START_CHECKPOINT_PATH='gs://train_with_tpu/networks_aydao_converted/sq-512'

!./tpu_config.sh

+ python run_training.py --num-gpus=8 --data-dir gs://train_with_tpu/datasets --config=config-f --dataset=sq-512 --gamma=5.0 --mirror-augment=false --metrics=none


attach_debugger <function attach_debugger at 0x7f038128cdd0>
print_status <function print_status at 0x7f038128ce60>
freeze_forever <function freeze_forever at 0x7f038128d050>
quit <function quit at 0x7f038128d0e0>
save_and_quit <function save_and_quit at 0x7f038128d200>
throw_exception <function throw_exception at 0x7f038128d290>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


attach_debugger <function attach_debugger at 0x7f0380e10b00>
print_status <function print_status at 0x7f0380e10b90>
freeze_forever <func

In [ ]:
!python convert_ckpt_to_pkl.py --tpu_address={tpu_address} --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-512 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

In [16]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl /content/drive/MyDrive/networks_aydao/model.ckpt-latest.pkl

Copying gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl...
/ [1 files][ 1007 MiB/ 1007 MiB]   51.0 MiB/s                                   
Operation completed over 1 objects/1007.6 MiB.                                   


In [ ]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-0* /content/drive/MyDrive/networks_aydao/

Copying gs://train_with_tpu/networks_aydao/model.ckpt-0.pkl...
| [1 files][995.8 MiB/995.8 MiB]   65.8 MiB/s                                   
Operation completed over 1 objects/995.8 MiB.                                    


In [ ]:
!gsutil cp -r gs://train_with_tpu/models/sq-256/model.ckpt-1024* /content/drive/MyDrive/networks_aydao/sq-256/

In [ ]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/sq-256/* /content/drive/MyDrive/networks_aydao/sq-256/

In [ ]:
!gsutil cp -r /content/drive/MyDrive/networks_aydao/sq-256/* gs://train_with_tpu/networks_aydao/sq-256/

In [ ]:
!pip install gcsfs

In [ ]:
!gcloud config set --help

In [ ]:
!gcloud config set project encoded-phalanx-326615

Updated property [core/project].


In [ ]:
#!export TPU_HOST=grpc://10.120.9.58:8470 # placeholder -- refer to shawwn's tpu code
#!export TPU_NAME=grpc://10.120.9.58:8470 # placeholder -- refer to shawwn's tpu code

!python convert_ckpt_to_pkl.py --tpu_address={tpu_address} --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-256 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
!python convert_ckpt_to_pkl.py --ckpt_model_dir /content/drive/MyDrive/networks_aydao/sq-256 --reference_pkl /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-256 --input_pkl gs://train_with_tpu/networks_aydao/model.ckpt-0.pkl

In [ ]:
!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-256 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl --kimg 5268480

In [ ]:
!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-512 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl --kimg 5268480

In [ ]:
sess = tf.Session('grpc://10.120.9.58:8470')
with sess:


In [ ]:
!pip3.6 install tensorflow-gpu==1.15.0

In [ ]:
!pip3.6 install tensorflow==1.15

In [ ]:
import tensorflow.compat.v2 as tf

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
#!export TPU_NAME={tpu_address}
#!export RESUME_PKL="/content/drive/MyDrive/sg2-ext_aydao-anime-danbooru2019s-512-5268480.pkl"
#!export RESUME_PKL=./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl
#!export RESUME_KIMG=100.4
#!export RESUME_TIME=43
#!export RESUME_TIME=`math 10*3600 + 13*60 + 49` # 10h 13m 49s
#RESUME_TIME=0*3600 + 44*60 + 31
#!export RESUME_TIME={RESUME_TIME} # 10h 13m 49s

#    resume_pkl="./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl"
#    resume_kimg=100.4
#    resume_time=0*3600 + 44*60 + 31
#    resume_pkl="/content/drive/MyDrive/stylegan2-cat-config-a.pkl"
#    resume_kimg=0
#    resume_time=0

config="config-f" # StyleGAN 2
#config="config-a" # StyleGAN 1

data_dir="/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery/datasets"
dataset="sq-512"
mirror="False"
metrics="none"
#!export NO_GCE_CHECK=true
total_kimg=5268480+1000
resume="/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl"
#resume="./results/00040-stylegan2-sq-512-1gpu-config-a"

!python run_training.py --resume "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"


### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.